In [13]:
#!pip install snowflake-connector-python==2.7.9

In [1]:
import snowflake
import snowflake.connector
import pandas as pd

## Connect to database

In [2]:
conn = snowflake.connector.connect(user = 'SVC_BEERTECH_CANDIDATE',
                                   password = '!uFbtIPCq53Ia5Er08*Ruc&blBPEei', 
                                   account = 'abinbev_naz.east-us-2.azure',
                                   warehouse = 'DEV_ELT_ADF_US_XS',
                                   database = 'candidate_sandbox',
                                   schema = 'ds_source')

## Find two wholesalers in NY & TX with one of top 10 highest unit sales since 2018

In [67]:
sql_sales = """
            select store.GEOGRAPHY_KEY,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            
            group by 1
            order by sum(UNIT_SALES) desc
            limit 10;
            
           """

pd.read_sql(sql_sales, conn)

,GEOGRAPHY_KEY,TOTAL_SALES
0,16568147,2.198203e+06
1,252791,1.584977e+06
2,662205,1.565420e+06
3,233057,1.485250e+06
4,134584,1.400589e+06
5,120262,1.388554e+06
6,199504,1.374629e+06
7,181530,1.238046e+06
8,258891,1.192355e+06
9,32596,1.167116e+06


- **TX - 255718** 
- **NY - 258891** 

## Find five products in NY & TX 

In [88]:
# 5 popular products in NY

sql_sales = """
            select s.PRODUCT_KEY,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            and store.GEOGRAPHY_KEY in (258891)
            
            group by 1
            order by sum(UNIT_SALES) desc
            limit 15;
            
           """

pd.read_sql(sql_sales, conn)

,PRODUCT_KEY,TOTAL_SALES
0,36849957,52794.0
1,680031864,19393.0
2,28488051,16881.0
3,659550424,11902.0
4,1413054,11049.0
5,1321133691,7513.0
6,210281273,7083.0
7,1400451,6853.0
8,154157221,6832.0
9,609521221,6028.0


In [89]:
# in Texas that also sell the popular products from NY

sql_sales = """
            select s.PRODUCT_KEY,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('TX')
            and store.GEOGRAPHY_KEY in (255718)
            and s.PRODUCT_KEY in (1321133691, 659550424, 1412422, 262793636,  609521221)
            
            group by 1
            order by sum(UNIT_SALES) desc;
            
           """

pd.read_sql(sql_sales, conn)

,PRODUCT_KEY,TOTAL_SALES
0,659550424,19413.0
1,1412422,11409.0
2,1321133691,6040.0
3,609521221,4153.0
4,262793636,1274.0


## Now, load data for the selected 2 wholesalers and 5 products from TX

In [102]:
sql_sales_tx = """
            select
            s.*, 
            to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy') as weekly_date,
            store.*,
            prod.*
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('TX')
            and store.GEOGRAPHY_KEY in (255718)
            and s.PRODUCT_KEY in (1321133691, 659550424, 1412422, 262793636,  609521221);
            
           """

tx_df = pd.read_sql(sql_sales_tx, conn)
tx_df.head()

,PRODUCT_KEY,GEOGRAPHY_KEY,TIME_KEY,UNIT_SALES,DOLLAR_SALES,VOLUME_SALES,WEEKLY_DATE,GEOGRAPHY_KEY,GEOGRAPHY_DESCRIPTION,WHOLESALER_ID_VALUE,...,ZIP_CODE,OUTLET_NAME,PRODUCT_KEY,PRODUCT_DESCRIPTION,AB_CATEGORY_STYLE_VALUE,AB_WAMP_VALUE,BEER_STYLE_VALUE,BRAND_VALUE,BREWER_VALUE,TYPE_VALUE
0,1412422,255718,2051,47.0,694.66,23.5,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,1412422,**********,EASY DRINKING,PREMIUM,TROPICAL LAGER,**********,CONSTELLATION BRANDS,BEER
1,609521221,255718,2051,18.0,233.46,9.0,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,609521221,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
2,1321133691,255718,2051,27.0,350.19,13.5,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,1321133691,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
3,659550424,255718,2051,77.0,998.69,38.5,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,659550424,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,MARK ANTHONY BRANDS INC,MALT BEVERAGE
4,262793636,255718,2051,7.0,111.93,4.4,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,262793636,**********,CRAFT & STYLES,PREMIUM,IPA,**********,FOUNDERS BREWING CO,BEER


## Now, load data for the selected wholesaler and 5 products from NY

In [103]:
sql_sales_ny = """
            select
            s.*, 
            to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy') as weekly_date,
            store.*,
            prod.*
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            and store.GEOGRAPHY_KEY in (258891)
            and s.PRODUCT_KEY in (1321133691, 659550424, 1412422, 262793636,  609521221);
            
           """

ny_df = pd.read_sql(sql_sales_ny, conn)
ny_df.head()

,PRODUCT_KEY,GEOGRAPHY_KEY,TIME_KEY,UNIT_SALES,DOLLAR_SALES,VOLUME_SALES,WEEKLY_DATE,GEOGRAPHY_KEY,GEOGRAPHY_DESCRIPTION,WHOLESALER_ID_VALUE,...,ZIP_CODE,OUTLET_NAME,PRODUCT_KEY,PRODUCT_DESCRIPTION,AB_CATEGORY_STYLE_VALUE,AB_WAMP_VALUE,BEER_STYLE_VALUE,BRAND_VALUE,BREWER_VALUE,TYPE_VALUE
0,1412422,258891,2051,22.0,318.78,11.0,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,1412422,**********,EASY DRINKING,PREMIUM,TROPICAL LAGER,**********,CONSTELLATION BRANDS,BEER
1,609521221,258891,2051,32.0,479.68,16.0,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,609521221,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
2,1321133691,258891,2051,33.0,494.67,16.5,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,1321133691,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
3,659550424,258891,2051,30.0,448.70,15.0,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,659550424,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,MARK ANTHONY BRANDS INC,MALT BEVERAGE
4,262793636,258891,2051,22.0,373.78,13.8,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,262793636,**********,CRAFT & STYLES,PREMIUM,IPA,**********,FOUNDERS BREWING CO,BEER


In [110]:
ny_df.shape, tx_df.shape

((1292, 23), (1242, 23))

## Cache data 

In [113]:
ny_df.to_csv('data/ny_df.csv', index=False)
tx_df.to_csv('data/tx_df.csv', index=False)